In [1]:
from src.ot_annotator2 import OTAnnotator
import time
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
from hyperopt import hp
import stlearn as st
import sys
import warnings
import os

warnings.filterwarnings("ignore")
np.random.seed(42) 

2025-01-02 10:48:23.812554: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-02 10:48:23.990035: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-01-02 10:48:23.990070: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-01-02 10:48:25.706213: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-

In [2]:
# output folder
out_folder = "tab_out2"
if not os.path.exists(out_folder):
    os.makedirs(out_folder, exist_ok=True)
else:
    print(f"Folder already exists: {out_folder}")


Folder already exists: tab_out2


In [3]:
# input data
dataset_paths = {
  '0027292__Region_1__20240530__125814' : "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027292__Region_2__20240530__125814' : "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027292__Region_3__20240530__125814' : "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027292__Region_4__20240530__125814' : "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027291__Region_1__20240530__125814' : "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027291__Region_2__20240530__125814' : "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027291__Region_3__20240530__125814' : "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027291__Region_4__20240530__125814' : "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027119__Region_1__20240621__120943': "/data/lemsaraa/amina/ST/ourData/20240621__120000__ST014_X0071_X0072_A4413/output-XETG00046__",
  '0027119__Region_2__20240621__120943': "/data/lemsaraa/amina/ST/ourData/20240621__120000__ST014_X0071_X0072_A4413/output-XETG00046__",
  '0027120__Region_1__20240621__120943':"/data/lemsaraa/amina/ST/ourData/20240621__120000__ST014_X0071_X0072_A4413/output-XETG00046__",
  '0027120__Region_2__20240621__120943': "/data/lemsaraa/amina/ST/ourData/20240621__120000__ST014_X0071_X0072_A4413/output-XETG00046__"
}

In [4]:
# hyperopt parameter space
# param_space = {
#             "reg": hp.loguniform("reg", np.log(0.0001), np.log(1)),
#             "reg_m_kl_1": hp.loguniform("reg_m_kl_1", np.log(0.0001), np.log(1)), #hp.uniform("reg_m_kl_1", 0, 500),
#             "reg_m_kl_2":hp.loguniform("reg_m_kl_2", np.log(0.0001), np.log(1)),# hp.uniform("reg_m_kl_2", 0, 500),
#             "method": hp.choice("method", ['sinkhorn']),
#             "reg_type": hp.choice("reg_type", ['entropy'])
#         }

param_space = {
            "reg": hp.loguniform("reg", np.log(0.0001), np.log(1)),
            "reg_m_kl_1": hp.loguniform("reg_m_kl_1", np.log(0.0001), np.log(10)), #hp.uniform("reg_m_kl_1", 0, 500),
            "reg_m_kl_2": hp.loguniform("reg_m_kl_2", np.log(0.0001), np.log(10)),# hp.uniform("reg_m_kl_2", 0, 500),
            "method": hp.choice("method", ['sinkhorn']),
            "reg_type": hp.choice("reg_type", ['kl'])
        }


# Load scRNAseq reference data with cell annotation

In [5]:
metadata=pd.read_csv('/data/lemsaraa/amina/ST/Xenium_analysis/tab/sc_metadata.csv',index_col=0)
metadata.head()

samples = {
    "BDP1083": "/data/lemsaraa/amina/ST/ourData/sc/06092024/cellranger/BDP1083_filtered_seurat_comp.h5",
    "BDP1105": "/data/lemsaraa/amina/ST/ourData/sc/06092024/cellranger/BDP1105_filtered_seurat_comp.h5",
    "BDP1130": "/data/lemsaraa/amina/ST/ourData/sc/06092024/cellranger/BDP1130_filtered_seurat_comp.h5",
    "BDP1131": "/data/lemsaraa/amina/ST/ourData/sc/06092024/cellranger/BDP1131_filtered_seurat_comp.h5",
}
adatas = {}
for sample_id, path in samples.items():
    sample_adata = sc.read_10x_h5(path)
    sample_adata.obs_names= sample_id+'_'+sample_adata.obs_names
    sample_adata.var_names_make_unique()
    adatas[sample_id] = sample_adata

adata_ref = ad.concat(adatas, label="sample")
adata_ref=adata_ref[metadata.index,:]
adata_ref.obs=metadata

# Cell Annotation (per cell)

In [6]:
task = 'Unbalanced_evaluation'
way = 'all'
verbose = True
nb_cluster = 10
metric = 'cosine'
op_iter = 100
key_ref = 'celltype_l2'
key_tar = 'leiden'
res = pd.DataFrame()

# Open a log file and redirect standard output
log_file = open(f"{out_folder}/{task}_verbose_matchit_{way}_{key_ref}.log", "w")
sys.stdout = log_file  

if verbose:
    print(f"""
    ## Configuration:
    Task = {task}
    Mapping Strategy = {way}
    Distance Metric = {metric}
    Reference Annotation = {key_ref}
    Target Clustering = {key_tar}
    Number of Clusters = {nb_cluster}        
    HyperOPT Iterations = {op_iter}
    """)
cellt=['','Pod_Injured','Pod_mut4']
# Iterate through datasets and paths
for sample in cellt:
    print(f"celltype excluded: {sample}")
    adata_ref_ = adata_ref[adata_ref.obs['celltype_l2'] != sample, :]
    
    for DATASET, DATA_DIR in dataset_paths.items():
        dataset_path = DATA_DIR + DATASET + "/"
        print(dataset_path)
        
        start_time = time.time()
        print(f"Start time: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")
        
        # Read Xenium dataset
        adata = st.ReadXenium(
            feature_cell_matrix_file=dataset_path + "cell_feature_matrix.h5",
            cell_summary_file=dataset_path + "cells.csv.gz"
        )
        annotation=pd.read_csv(out_folder+"/CellAnnotation_"+DATASET+"_matchit_all_celltype_l2.csv",index_col=0)
        annotation= annotation[annotation["predicted_annotation"].map(annotation["predicted_annotation"].value_counts()) > 1]    
        adata=adata[annotation.index,:]
        adata.obs['predicted_annotation']=annotation['predicted_annotation']
        adata=adata[adata.obs['predicted_annotation'].isin(['Pod_Injured','Pod_mut4','Pod']),:]
        # Exclude the current sample from the reference data
        
        # Identify genes of interest
        gene_interest = adata_ref.var_names.intersection(adata.var_names)
        
        # Initialize OTAnnotator
        annotator = OTAnnotator(
            adata, 
            adata_ref_, 
            gene_interest, 
            param_space=param_space,
            key_ref=key_ref, 
            key_tar=key_tar, 
            way=way, 
            verbose=verbose
        )
        
        # Perform subclustering and mapping
        print('*** Step 1: Subcluster Reference and Target ***') 
        annotator.subcluster(nb_cluster=nb_cluster)
        
        print('*** Step 2: Mapping ***')
        annotator.annotate(op_iter=op_iter, metric=metric)
        
        print("Best Parameters:", annotator.best_params)
        
        # Log execution details
        end_time = time.time()
        print(f"End time: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time))}")
        print(f"Execution time: {end_time - start_time:.2f} seconds")
        
        # Save results to the DataFrame
        res = pd.concat([res, pd.DataFrame([{
            'dataset': DATASET,
            'sample': sample,
            'reg_m_kl_1': annotator.best_params['reg_m_kl_1'],
            'reg_m_kl_2': annotator.best_params['reg_m_kl_2']
        }])], ignore_index=True)

# Restore standard output and close the log file
print("\nEnded successfully.") 
sys.stdout = sys.__stdout__  
log_file.close()


In [7]:
res.to_csv(f"{out_folder}/{task}_matchit_{way}.csv")